In [2]:
from dashboard.logic.io import GSHEETS_URL, read_gsheet, comment_button
import pandas as pd
import numpy as np

df = read_gsheet(
    GSHEETS_URL, 
    header=None
)

In [3]:
def findRowColRegex(df: pd.DataFrame, pat: str, case: bool=True, regex:bool=True):
    
    # make sure only one such pat exists in the df
    df_mask = df.apply(lambda x: x.str.contains(pat, case=case, regex=regex) if x.dtype == 'object' else None)
    
    count = df_mask.sum().sum()
    
    if count < 1: 
        raise ValueError(f'Given {pat} did not give any results.')
    if count > 1:
        raise ValueError(f'Given {pat} gave more than 1 results.')
     
    # col and row values where pat
    row = df_mask.any(axis='columns').argmax()
    col = df_mask.any(axis='index').argmax()
    
    return row, col

def getDataFrames(df: pd.DataFrame):

    df = df.reset_index(drop=True)
    dataframes_dict = {}
    
    # MAIN DF
    main_df_idx2 = (df == 'Monthly Income').any(axis='columns').argmax()
    dataframes_dict['main'] = df.iloc[:main_df_idx2+1,:3].dropna(how='all', axis='rows')
    
    # ADS DF
    r1, c1 = findRowColRegex(df, "My Finance Course")
    
    
    ads_df = df.iloc[r1:,c1:].reset_index(drop=True)
    ads_df = (ads_df
        .loc[:(ads_df[c1].isna()).argmax()-1, :]
        .dropna(how='all', axis='columns')
    )
    dataframes_dict['ads'] = ads_df
    
    # OTHER DFS
    df_names = [
    'announcements',
    'advice',
    'error_warning',
    'risk',
    'skip',
    'skip',
    'skip',
    'historical',
    'cash_pos',
    'general_notes',
    'success',
    ]
    
    # Find rows after main df where all values are NaN-s.
    nan_rows = df.iloc[main_df_idx2+1:,].isna().all(axis='columns')
    
    # Indices where all NaN-s in the row
    dfs_idxs = []
    
    for i,bool in nan_rows.items():
        
        if i == nan_rows.index[-1]:
            dfs_idxs.append(None)
            break
        if bool:
            dfs_idxs.append(i)
    
    # raw dataframes
    dfs = []        
    for i,idx in enumerate(dfs_idxs):
        if i == len(dfs_idxs) - 1:
            break
        dfs.append(df.iloc[idx:dfs_idxs[i+1]])
    
    # strip dataframes from NaN-s and add to dictionary
    for df_, name in zip(dfs, df_names):
        
        if name == 'skip': continue
        
        dataframes_dict[name] = (df_
            .dropna(how='all', axis='columns')
            .dropna(how='all', axis='rows')
        )
    
    return dataframes_dict

df1 = df.copy()

#findRowColRegex(df1, 'My Finance Course')

df_dict = getDataFrames(df1)
df_dict.keys()

dict_keys(['main', 'ads', 'announcements', 'advice', 'error_warning', 'risk', 'historical', 'cash_pos', 'general_notes', 'success'])

In [10]:
df_ads = df_dict['ads'].copy()
df_ads.columns = ['text', 'hyperlink']
df_ads['icon'] = df_ads.text.str[:1]
df_ads['text'] = df_ads.text.str[2:]

headers = ['Course', 'Mentoring', 'PM Global', 'PM USA', 'PM UK', 'Crypto Security', 'Stock Platform', 'Global Bank Account']

df_ads['header'] = headers
df_ads

,text,hyperlink,icon,header
0,My Finance Course,https://bit.ly/3qzW4ZB,📈,Course
1,Private 1-on-1 Sessions With Me: (1 hour slots),https://bit.ly/3bxv3xN,🧠,Mentoring
2,Where I Buy Allocated Silver/Gold Globally:,https://bit.ly/3kbgO61,🪙,PM Global
3,Where I Buy Physical Silver/Gold in the USA:,https://bit.ly/3drNR2F,🇺,PM USA
4,Where I Buy Physical Silver/Gold in the UK:,https://bit.ly/3SFvp9E,🇬,PM UK
5,How I Protect My Crypto from hackers:,https://bit.ly/3bK8oyL,🔑,Crypto Security
6,The Stock Platform I use for Europe (FREE Cred...,https://bit.ly/3KoufvK,📈,Stock Platform
7,A Global bank account (that I use) that offers...,https://bit.ly/3JpWOZK,🏦,Global Bank Account


In [17]:
for k,v in df_ads.items():
    print(type(k))
    break

<class 'str'>


In [17]:
idx_break = int(df_ads.shape[0] // 2)
indices = [(None, idx_break), (idx_break,  None)]
#for half in range():
df_ads[4:None]

,text,hyperlink
4,🇬🇧 Where I Buy Physical Silver/Gold in the UK:,https://bit.ly/3SFvp9E
5,🔑 How I Protect My Crypto from hackers:,https://bit.ly/3bK8oyL
6,📈 The Stock Platform I use for Europe (FREE Cr...,https://bit.ly/3KoufvK
7,🏦 A Global bank account (that I use) that offe...,https://bit.ly/3JpWOZK


In [20]:
[i for i in range(2, 0)]

[]